This is a solid architecture for a lightweight ML application. Migrating from Jupyter Notebooks to a cloud architecture involves moving from "interactive exploration" to "stateless scripts."

Here is your step-by-step migration plan, mapping your specific notebooks to the AWS components in your diagram.

---

### Phase 1: Prepare the Model Code (The "EC2" Part)

Your diagram shows an **EC2 instance** used for "Model Training." You currently have 6 notebooks. You don't need all of them for production. You only need the **best performing pipeline**.

Based on your notebooks, **NB6 (Comparison)** and **NB5 (Optimizing SVM)** contain the final logic.

**1. Consolidate Logic into `train.py`**
You need to create a single Python script (`train.py`) that runs on the EC2 instance. This script will replace the manual running of notebooks.

*   **Input:** `data/clean-data.csv` (or raw data).
*   **Logic to extract:**
    *   **From NB3/NB6:** The `Pipeline` creation. *Crucial:* You must use a Pipeline (StandardScaler + PCA + SVC) so that raw input data is automatically scaled and transformed during prediction.
    *   **From NB5:** The best hyperparameters you found (e.g., `C`, `gamma`, `kernel`).
*   **Output:** A serialized file (`model_v1.pkl`).

**Code Snippet for `train.py` (to run on EC2):**


In [ ]:
import pandas as pd
import boto3
import joblib
import os
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# 1. Load Data (Ideally from S3, but locally on EC2 for now)
DATA_PATH = 'data/clean-data.csv'
if not os.path.exists(DATA_PATH):
    os.makedirs('data', exist_ok=True)
    # Try downloading from S3 if available
    source_bucket = 'YOUR_SOURCE_DATA_BUCKET_NAME'
    source_key = 'clean-data.csv'
    try:
        s3_tmp = boto3.client('s3')
        s3_tmp.download_file(source_bucket, source_key, DATA_PATH)
        print(f"Downloaded {DATA_PATH} from s3://{source_bucket}/{source_key}")
    except Exception as e:
        print(f"Warning: Could not download from S3: {e}")
        # Ensure file exists or raise error
        if not os.path.exists(DATA_PATH):
             raise FileNotFoundError(f"{DATA_PATH} not found locally and S3 download failed.")

data = pd.read_csv(DATA_PATH)

# --- Data Cleaning & Preprocessing (Replicating NB1 & NB3) ---
# 1. Drop 'Unnamed: 0' (Artifact from saving CSV with index)
if 'Unnamed: 0' in data.columns:
    data.drop('Unnamed: 0', axis=1, inplace=True)

# 2. Drop 'id' if it exists (Redundant - from NB1)
if 'id' in data.columns:
    data.drop('id', axis=1, inplace=True)

# 3. Drop 'Unnamed: 32' if it exists (Common dataset artifact)
if 'Unnamed: 32' in data.columns:
    data.drop('Unnamed: 32', axis=1, inplace=True)

# 4. Separate Features and Target
# After cleaning, 'diagnosis' is the first column (index 0)
X = data.iloc[:, 1:].values # Features (columns 1 to end)
y = data.iloc[:, 0].values  # Target (column 0)

# 5. Encode Target (M=1, B=0) - from NB3
le = LabelEncoder()
y = le.fit_transform(y)

# 2. Define the Pipeline (Logic from NB6)
# Use the best params found in NB5
pipeline = Pipeline([
    ('scl', StandardScaler()),
    ('pca', PCA(n_components=2)),
    ('clf', SVC(C=100.0, kernel='rbf', probability=True)) 
])

# 3. Train
pipeline.fit(X, y)

# 4. Save Model locally
joblib.dump(pipeline, 'model_v1.pkl')

# 5. Upload to S3 (The "Stores Model" bucket in your diagram)
try:
    s3 = boto3.client('s3')
    s3.upload_file('model_v1.pkl', 'YOUR_MODEL_BUCKET_NAME', 'latest_model.pkl')
    print("Model uploaded to S3")
except Exception as e:
    print(f"Warning: Could not upload to S3 (Check credentials/bucket name): {e}")

FileNotFoundError: data/clean-data.csv not found locally and S3 download failed: Unable to locate credentials

### Phase 1.5: Environment Setup on EC2

You mentioned you have a `.yaml` file ready. This is perfect. Here is the exact sequence of commands to set up your EC2 instance to run the `train.py` script.

**1. Connect to your EC2 Instance**
Use SSH or EC2 Instance Connect.

**2. Install Micromamba (Faster than Conda)**
Run these commands in your EC2 terminal to install the package manager:
```bash
"${SHELL}" <(curl -L micro.mamba.pm/install.sh)
source ~/.bashrc
```

**3. Upload your Files**
Upload `breastcancer_env.yaml` and `train.py` to the instance (you can use `scp` or VS Code Remote - SSH).

**4. Create the Environment**
```bash
micromamba create -f breastcancer_env.yaml
micromamba activate breastcancer
```

**5. Verify Dependencies**
Since `boto3` (AWS SDK) is required for S3 access but might not be in your original data science YAML, ensure it is installed:
```bash
micromamba install boto3
```

**6. Run the Training Script**
```bash
python train.py
```



**Action Item:** Launch an EC2 instance (t3.medium is fine), install your environment using your `.yaml` file, run this script, and ensure `latest_model.pkl` appears in your S3 bucket.

---

### Phase 2: The Predictor (The "Lambda" Part)

This is the "AWS Lambda: Predictor" in your diagram. It needs to load the model from S3 and make a prediction.

**Challenge:** AWS Lambda has size limits. `pandas` and `scikit-learn` are heavy.
**Solution:** Use a **Lambda Container Image** (Docker) or use **AWS Data Wrangler Layers**.

**Logic for `lambda_function.py`:**


In [ ]:
import json
import boto3
import joblib
import os
import numpy as np

# Initialize clients outside handler for caching
s3 = boto3.client('s3')
dynamodb = boto3.resource('dynamodb')
sns = boto3.client('sns')
table = dynamodb.Table('YOUR_DYNAMODB_TABLE_NAME')
TOPIC_ARN = 'YOUR_SNS_TOPIC_ARN'

# Download model to /tmp (Lambda's only writable storage)
s3.download_file('YOUR_MODEL_BUCKET_NAME', 'latest_model.pkl', '/tmp/model.pkl')
model = joblib.load('/tmp/model.pkl')

def lambda_handler(event, context):
    # 1. Parse Input (from API Gateway)
    body = json.loads(event['body'])
    # Expecting a list of 30 features
    features = np.array(body['data']).reshape(1, -1)
    
    # 2. Predict
    prediction = model.predict(features)[0] # 0 or 1
    probability = model.predict_proba(features)[0].max()
    
    result = "Malignant" if prediction == 1 else "Benign"
    
    # 3. Store in DynamoDB
    item = {
        'patient_id': body.get('patient_id', 'unknown'),
        'features': str(body['data']),
        'prediction': result,
        'confidence': str(probability),
        'status': 'Pending Validation' # For your feedback loop
    }
    table.put_item(Item=item)
    
    # 4. SNS Notification (If Malignant)
    if prediction == 1:
        sns.publish(
            TopicArn=TOPIC_ARN,
            Message=f"Alert: High risk patient detected. ID: {item['patient_id']}",
            Subject="Breast Cancer Risk Alert"
        )
        
    return {
        'statusCode': 200,
        'body': json.dumps({'prediction': result, 'confidence': probability})
    }



---

### Phase 3: The Retraining Loop (EventBridge + Lambda)

This is the "AWS Lambda: Retraining" in your diagram.

**How to adjust data location:**
In your notebooks, you read from CSV. In this Lambda, you must read from **DynamoDB** (where you are storing new patient records) or a "Historical Data" folder in **S3**.

**Logic:**
1.  Triggered by EventBridge (e.g., every Sunday).
2.  Fetch historical data + new "Confirmed" data from DynamoDB.
3.  Combine them into a DataFrame.
4.  Run the same training logic as Phase 1.
5.  Overwrite `latest_model.pkl` in S3.

*Note: If your dataset grows large, Lambda might time out (15 min limit). If that happens, you will need to move this logic back to EC2 or AWS SageMaker, but for < 10,000 rows, Lambda is fine.*

---

### Phase 4: Infrastructure Setup Checklist

1.  **S3 Buckets:**
    *   `bucket-models`: To store `latest_model.pkl`.
    *   `bucket-website`: To host your HTML/JS frontend (if applicable).

2.  **DynamoDB Table:**
    *   Partition Key: `patient_id` (String).
    *   Attributes: `prediction`, `features`, `actual_diagnosis` (for the feedback loop).

3.  **SNS Topic:**
    *   Create a topic.
    *   Subscribe your email address to it.

4.  **IAM Roles (The Glue):**
    *   **EC2 Role:** Needs `S3FullAccess` (to upload the model).
    *   **Lambda Role:** Needs `S3ReadOnly` (to load model), `DynamoDBFullAccess` (to save records), and `SNSPublish` (to send alerts).

### Summary of Code Migration

| Notebook Source | AWS Destination | Action |
| :--- | :--- | :--- |
| **NB1, NB2, NB3** (EDA/Cleaning) | **N/A (One-off)** | These are for your understanding. The *cleaning logic* (e.g., dropping IDs) must be moved into the `train.py` script. |
| **NB6** (Pipeline Definition) | **EC2 `train.py`** | Copy the `Pipeline(...)` definition exactly. This ensures your training and inference match. |
| **NB6** (Model Saving) | **EC2 `train.py`** | Add `joblib.dump` to save the file to S3. |
| **NB4** (Prediction Logic) | **Lambda (Predictor)** | The `clf.predict()` line goes here. |
| **New Logic** | **Lambda (Retraining)** | Logic to pull data from DynamoDB and re-run `pipeline.fit()`. |

### How to start today?
1.  **Local Test:** Create the `train.py` script on your laptop. Run it. Ensure it produces a `.pkl` file.
2.  **Local Inference:** Write a small script that loads that `.pkl` file and predicts on a dummy array of 30 numbers.
3.  **AWS Upload:** Once those two work locally, spin up the EC2 and move `train.py` there.

### Phase 3: The Retraining Loop (EventBridge + Lambda)

This is the "AWS Lambda: Retraining" in your diagram.

**How to adjust data location:**
In your notebooks, you read from CSV. In this Lambda, you must read from **DynamoDB** (where you are storing new patient records) or a "Historical Data" folder in **S3**.

**Logic:**
1.  Triggered by EventBridge (e.g., every Sunday).
2.  Fetch historical data + new "Confirmed" data from DynamoDB.
3.  Combine them into a DataFrame.
4.  Run the same training logic as Phase 1.
5.  Overwrite `latest_model.pkl` in S3.

*Note: If your dataset grows large, Lambda might time out (15 min limit). If that happens, you will need to move this logic back to EC2 or AWS SageMaker, but for < 10,000 rows, Lambda is fine.*

### Phase 4: Infrastructure Setup Checklist

1.  **S3 Buckets:**
    *   `bucket-models`: To store `latest_model.pkl`.
    *   `bucket-website`: To host your HTML/JS frontend (if applicable).

2.  **DynamoDB Table:**
    *   Partition Key: `patient_id` (String).
    *   Attributes: `prediction`, `features`, `actual_diagnosis` (for the feedback loop).

3.  **SNS Topic:**
    *   Create a topic.
    *   Subscribe your email address to it.

4.  **IAM Roles (The Glue):**
    *   **EC2 Role:** Needs `S3FullAccess` (to upload the model).
    *   **Lambda Role:** Needs `S3ReadOnly` (to load model), `DynamoDBFullAccess` (to save records), and `SNSPublish` (to send alerts).

### Summary of Code Migration

| Notebook Source | AWS Destination | Action |
| :--- | :--- | :--- |
| **NB1, NB2, NB3** (EDA/Cleaning) | **N/A (One-off)** | These are for your understanding. The *cleaning logic* (e.g., dropping IDs) must be moved into the `train.py` script. |
| **NB6** (Pipeline Definition) | **EC2 `train.py`** | Copy the `Pipeline(...)` definition exactly. This ensures your training and inference match. |
| **NB6** (Model Saving) | **EC2 `train.py`** | Add `joblib.dump` to save the file to S3. |
| **NB4** (Prediction Logic) | **Lambda (Predictor)** | The `clf.predict()` line goes here. |
| **New Logic** | **Lambda (Retraining)** | Logic to pull data from DynamoDB and re-run `pipeline.fit()`. |

### How to start today?
1.  **Local Test:** Create the `train.py` script on your laptop. Run it. Ensure it produces a `.pkl` file.
2.  **Local Inference:** Write a small script that loads that `.pkl` file and predicts on a dummy array of 30 numbers.
3.  **AWS Upload:** Once those two work locally, spin up the EC2 and move `train.py` there.

In [ ]:
import boto3
import pandas as pd
import joblib
import os
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC

# Initialize clients
s3 = boto3.client('s3')
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('YOUR_DYNAMODB_TABLE_NAME')

def lambda_handler(event, context):
    # 1. Fetch Data from DynamoDB (New Data)
    # Scan is expensive, in production use Query or export to S3
    response = table.scan()
    items = response['Items']
    
    # Convert to DataFrame
    new_data = pd.DataFrame(items)
    
    # 2. Fetch Historical Data (from S3)
    s3.download_file('YOUR_DATA_BUCKET', 'historical_data.csv', '/tmp/historical.csv')
    historical_data = pd.read_csv('/tmp/historical.csv')
    
    # 3. Combine Data
    # Ensure columns match. You might need to parse 'features' string back to columns
    # This is a simplified example
    # full_data = pd.concat([historical_data, new_data])
    
    # For this example, we'll just retrain on historical to show the logic
    X = historical_data.iloc[:, 1:31].values
    y = historical_data.iloc[:, 0].values
    
    # 4. Retrain Model
    pipeline = Pipeline([
        ('scl', StandardScaler()),
        ('pca', PCA(n_components=2)),
        ('clf', SVC(C=100.0, kernel='rbf', probability=True)) 
    ])
    pipeline.fit(X, y)
    
    # 5. Save and Upload New Model
    joblib.dump(pipeline, '/tmp/model_v2.pkl')
    s3.upload_file('/tmp/model_v2.pkl', 'YOUR_MODEL_BUCKET_NAME', 'latest_model.pkl')
    
    return {
        'statusCode': 200,
        'body': 'Model Retrained and Updated Successfully'
    }